In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def load_mnist(path, kind='train'):
    import os

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte')

    with open(labels_path, 'rb') as labels_file:
        labels = np.frombuffer(labels_file.read(), dtype=np.uint8,
                               offset=8)

    with open(images_path, 'rb') as images_file:
        images = np.frombuffer(images_file.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 28, 28)

    return tf.convert_to_tensor(images), tf.convert_to_tensor(labels)

def filter_mnist(images, labels, classes):
    classes = tf.reshape(classes, [-1, 1])
    labels = tf.reshape(labels, [1, -1])

    eq = tf.equal(labels, classes)
    mask = tf.reduce_any(eq, 0)

    labels = tf.reshape(labels, [-1, 1])
    return tf.boolean_mask(images, mask), tf.boolean_mask(labels, mask)

def load_data(kind='train', path='zalando', classes=tf.constant([3, 5], dtype=tf.uint8)):
    images, labels = load_mnist(path, kind)
    return filter_mnist(images, labels, classes)

2021-10-18 22:39:10.125292: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_images, train_labels = load_data()
test_images, test_labels = load_data(kind='t10k')

In [ ]:
import matplotlib.pyplot as plt

img = 1.0 - train_images[0].numpy() / 255.0
plt.imshow(img, cmap='gray')

In [104]:
def normalize(images):
    casted = tf.cast(tf.reshape(images, (images.shape[0], -1)), dtype=tf.float32)
    norm = tf.linalg.norm(casted, axis=1)

    return casted / tf.reshape(norm, [-1, 1])

def map_labels(labels, zero_class=3):
    condition = tf.equal(labels, tf.constant(zero_class, shape=labels.shape))
    zeros = tf.zeros(labels.shape)
    ones = tf.ones(labels.shape)

    return tf.where(condition, zeros, ones)

In [106]:
from tensorflow import keras

## todo -> take it from here :)
class Linear(keras.layers.Layer):
    def __init__(self, input_dims, output_dims):
        super(Linear, self).__init__()

        w_init = tf.random_uniform_initializer(0.0, 1.0)
        self.w = tf.Variable(
            initial_value=w_init(shape=[input_dims, output_dims], dtype=tf.float32),
            trainable=True
        )

        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=[input_dims], dtype=tf.float32),
            trainable=True
        )

    def call(self, inputs):
            return tf.matmul(inputs, self.w) + self.b

In [105]:
classes = tf.constant([3, 5], dtype=tf.int32)
classes = tf.reshape(classes, [-1, 1])
labels = tf.constant([1, 3, 2, 5, 4])
labels = tf.reshape(labels, [1, -1])

res = tf.equal(labels, classes)
reduced = tf.reduce_any(res, 0)

res = map_labels(labels)
res



<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[1., 0., 1., 1., 1.]], dtype=float32)>